<a href="https://colab.research.google.com/github/GabrielMalinowski/GumtreeSpider/blob/master/Workshop_3_Paginacja.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Generatory, Yield

In [1]:
mylist = [0, 1, 4]
for i in mylist:
    print(i)

0
1
4


In [2]:
mylist = [x*x for x in range(3)] # lista cała w pamięci
for i in mylist:
    print(i)

0
1
4


In [3]:
mylist = (x*x for x in range(3)) # list comprehension
for i in mylist:
    print(i)

0
1
4


In [4]:
def createGenerator():
    mylist = range(3)
    for i in mylist:
        yield i*i # domknięcie (closure)
        
for i in createGenerator():
    print(i)

0
1
4


# Fail2Ban

As you can see in my example, I have set up 300 maxretry and 300 for findtime, so, we need to have 300 GETs from the same IP in a time window of 300 seconds to have the originating IP blocked.

# Scraper

Scrapy doesn’t wait a fixed amount of time between requests, but uses a random interval between 0.5 * DOWNLOAD_DELAY and 1.5 * DOWNLOAD_DELAY.

In [0]:
import scrapy
import scrapy.crawler as crawler
from bs4 import BeautifulSoup

from scrapy.crawler import CrawlerProcess

class MySpider(scrapy.Spider):
    name = 'myspider'
    start_urls = [
        'https://www.gumtree.pl/s-mieszkania-i-domy-sprzedam-i-kupie/mazowieckie/v1c9073l3200001p1'
        ]
   
    custom_settings = {
        'DOWNLOAD_DELAY': '4.0',
    }

    top_url = 'https://www.gumtree.pl'
    def parse(self, response):
        self.logger.info('Got successful response from {}'.format(response.url))
        soup = BeautifulSoup(response.body, 'lxml')
        link_tabs = soup.findAll("div", {"class": "result-link"})
        item_urls = []
        for tab in link_tabs:
            hrefs = tab.findAll("a", {"class": "href-link"})
            for h in hrefs:
                item_urls.append(self.top_url + h["href"])
            
        
        for item_url in item_urls:
            yield scrapy.Request(item_url, self.parse_item)

    def parse_item(self, response):
        self.logger.info('Got successful response from {}'.format(response.url))
        # <processing code not shown>
        #item = MyItem()
        # populate `item` fields
        # and extract item_details_url
        #yield scrapy.Request(item_details_url, self.parse_details, meta={'item': item})




In [0]:
process = CrawlerProcess({
    'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'
})
process.crawl(MySpider)
process.start()

In [0]:
import scrapy
import scrapy.crawler as crawler
from bs4 import BeautifulSoup

from scrapy.crawler import CrawlerProcess

class MyPaginatingSpider(scrapy.Spider):
    name = 'mypaginatingspider'
    start_urls = [
        'https://www.gumtree.pl/s-mieszkania-i-domy-sprzedam-i-kupie/mazowieckie/v1c9073l3200001p1'
        ]
   
    custom_settings = {
        'DOWNLOAD_DELAY': '6.0',
    }

    top_url = 'https://www.gumtree.pl'
    def parse(self, response):
        self.logger.info('Got successful response from {}'.format(response.url))
        soup = BeautifulSoup(response.body, 'lxml')
        link_tabs = soup.findAll("div", {"class": "result-link"})
        item_urls = []
        next_urls = []
        for tab in link_tabs:
            hrefs = tab.findAll("a", {"class": "href-link"})
            for h in hrefs:
                item_urls.append(self.top_url + h["href"])
            
        nexts = soup.findAll("a", {"class": "next"})
        
        for h in nexts:
            for h in hrefs:
                next_urls.append(self.top_url + h["href"])
        
        print (next_urls)
            
        for item_url in item_urls:
            yield scrapy.Request(item_url, self.parse_item)

        #for next_url in next_urls:
        #    yield scrapy.Request(item_url, self.parse)
    
            
    def parse_item(self, response):
        self.logger.info('Got successful response from {}'.format(response.url))



In [0]:
process = CrawlerProcess({
    'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'
})
process.crawl(MyPaginatingSpider)
process.start()